In [1]:
# Place your imports here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import datetime
from statsmodels.regression.rolling import RollingOLS
from scipy.stats.mstats import winsorize
from linearmodels.panel import PanelOLS
import wrds
import time


import warnings
warnings.filterwarnings('ignore')

In [10]:
class hw_ETF(object):
    """
    Class to replicate the flow sensitivity analysis in Goldstein et al. (2017).
    You will need to implement the methods in this class, wherever there is a TODO.

    """

    def __init__(self):
        self.fund_summary_df = None
        self.shares_df = None
        self.impute_basket_df = None
        self.merged_df = None

    def clean_fund_summary(self, filename):
        # for this part, you work with crsp_q_mutualfunds_summary_2019_2023.csv, which you can download with the codes below
        '''
        df_fund_summary_query = "SELECT * FROM crsp_q_mutualfunds.fund_summary2 WHERE (caldt BETWEEN '2019-01-01' AND '2023-12-31')"
        df = db.raw_sql(df_fund_summary_query)
        df.to_csv(data_path + "/crsp_q_mutualfunds_summary_2019_2023.csv", index=False)
        '''
        df = pd.read_csv(filename, encoding='ISO-8859-1', low_memory=False)
        df['caldt'] = pd.to_datetime(df['caldt'])

        exclude = '|'.join(['Treasury', 'U.S. Government', 'Municipal', 'Emerging Market', 'Emerging Mrkts', 'International', 'Global'])
        df = df[(df['et_flag'] == 'F') & (df['index_fund_flag'] == 'D') & 
                (df['crsp_obj_cd'] == 'I') & (~df['lipper_class_name'].str.contains(exclude, case=False, na=False)) &
                (~df['fund_name'].str.contains(exclude, case=False, na=False)) & (df['caldt'].dt.year == 2020)]

        df = df.drop_duplicates(subset='crsp_portno')

        self.fund_summary_df = df
        return df['crsp_portno'].dropna().astype(int).tolist(), df['ticker'].dropna().tolist()

    def clean_shares(self, filename):
        # for this part, you work with crsp_q_mutualfunds_shares_2020.csv
        # Import shares data: CRSP monthly stock file; shrout (Number of Shares Outstanding in thousands). Query ETF tickers above, 2019m12 - 2021m1 
        # the data is also available on Canvas
        shares_df = pd.read_csv(filename, parse_dates=['date'])
        shares_df.sort_values(by=['PERMNO', 'date'], inplace=True)

        # Calculate pct_change directly
        shares_df['etf_shares_change'] = shares_df.groupby('PERMNO')['SHROUT'].pct_change()
        shares_df['month'] = shares_df['date'].dt.month
        shares_df['year'] = shares_df['date'].dt.year

        self.shares_df = shares_df
        return shares_df['etf_shares_change'].dropna().tolist()

    def impute_baskets(self, filename):
        # for this part, you work with crsp_q_mutualfunds_holdings_2020.csv, which you can download with the codes below
        '''
        df_query_holdings = "SELECT * FROM crsp.holdings WHERE (report_dt BETWEEN '2019-12-01' AND '2021-01-31')"
        merged_df = db.raw_sql(df_query_holdings)
        merged_df.to_csv(f"{data_path}/crsp_q_mutualfunds_holdings_2020.csv", index=False)
        '''
        df = pd.read_csv(filename)
        df['cusip'].fillna(df['security_name'], inplace=True)
        df.sort_values(by=['crsp_portno', 'report_dt'], inplace=True)
        df['shares_change'] = df.groupby(['crsp_portno', 'cusip'])['nbr_shares'].diff()
        df['imputed_basket_weight_level'] = df.groupby(['crsp_portno', 'report_dt'])['shares_change'].transform(lambda x: abs(x).sum())
        
        # Calculate unique securities
        df = df.assign(
            portfolio_num_sec=df.groupby(['crsp_portno', 'report_dt'])['cusip'].transform('nunique'),
            basket_num_sec=df[df['shares_change'] != 0].groupby(['crsp_portno', 'report_dt'])['cusip'].transform('nunique')
        ).fillna({'imputed_basket_weight_level': 0, 'portfolio_num_sec': 0, 'basket_num_sec': 0})
        
        self.impute_basket_df = df
        return df['portfolio_num_sec'].tolist(), df['basket_num_sec'].tolist()
    
    
    def merge_summary_shares(self):
        merged = pd.merge(self.fund_summary_df, self.impute_basket_df, on='crsp_portno', how='inner')
        merged['report_dt'] = pd.to_datetime(merged['report_dt'])
        merged = merged.assign(year=merged['report_dt'].dt.year, month=merged['report_dt'].dt.month)

        merged.rename(columns={'ticker_x': 'ticker'}, inplace=True)
        merged.drop(columns=['ticker_y'], inplace=True)


        shares = self.shares_df.rename(columns={'TICKER': 'ticker'})
        shares = shares.dropna(subset=['ticker', 'year', 'month'])
        final_merged = pd.merge(merged, shares, on=['ticker', 'year', 'month'], how='inner')
        
        self.merged_df = final_merged
        return final_merged.head(5)
    

    def analysis(self):
        df = self.merged_df.drop_duplicates(subset=['ticker', 'month', 'year'])
        df = df[(df['portfolio_num_sec'] > 0) & (df['basket_num_sec'] > 0)]

        num_ETFs = df['ticker'].nunique()
        avg_bonds_portfolio = df['portfolio_num_sec'].mean()
        avg_bonds_basket = df['basket_num_sec'].mean()
        avg_basket_frac = (df['basket_num_sec'] / df['portfolio_num_sec']).mean()

        return num_ETFs, avg_bonds_portfolio, avg_bonds_basket, avg_basket_frac
    
    def analysis_verbal(self):
        '''
        Please type your response to the question: 
        For each ETF, calculate average number of bonds in the basket and in the ETF holdings. Divide the average number of bonds in baskets by the number of bonds in the ETF holdings. What do you observe?
        '''
        reponse = "Baskets are crucial in shaping monthly ETF portfolio adjustments, far beyond being mere representative samples. Their substantial share underscores the importance of basket composition in affecting the portfolio's structure and the liquidity of its bonds."
        return reponse

In [4]:
class hw_ETF(object):
    """
    Class to replicate the flow sensitivity analysis in Goldstein et al. (2017).
    You will need to implement the methods in this class, wherever there is a TODO.

    """

    def __init__(self):
        self.fund_summary_df = None
        self.shares_df = None
        self.impute_basket_df = None
        self.merged_df = None

    def clean_fund_summary(self, filename):
        # for this part, you work with crsp_q_mutualfunds_summary_2019_2023.csv, which you can download with the codes below
        '''
        df_fund_summary_query = "SELECT * FROM crsp_q_mutualfunds.fund_summary2 WHERE (caldt BETWEEN '2019-01-01' AND '2023-12-31')"
        df = db.raw_sql(df_fund_summary_query)
        df.to_csv(data_path + "/crsp_q_mutualfunds_summary_2019_2023.csv", index=False)
        '''
        df = pd.read_csv(filename, encoding='ISO-8859-1', low_memory=False)

        # Convert 'caldt' to datetime format to ensure correct filtering by year
        df['caldt'] = pd.to_datetime(df['caldt'])

        # Do the following manipulations with df:
            # Filter data: ETFs, Index funds, Fixed income
            # Exclude certain categories
            # Keep date range of interest: 2020
            # Drop duplicates in crsp_portno
            # Save processed dataset

        ### Filter fund types
        # Keep ETFs (assuming 'et_flag' indicates ETFs; adjust as needed)
        # Index funds (assuming 'index_fund_flag' indicates index funds; adjust as needed)
        # Fixed income funds, i.e., if the CRSP objective code is 'I'
        df_filtered = df[
            (df['et_flag'] == 'F') &  
            (df['index_fund_flag'] == 'D') &
            (df['crsp_obj_cd'] == 'I')
        ]

        # Exclude certain categories based on lipper_class_name and fund_name
        exclude_categories = ['Treasury', 'U.S. Government', 'Municipal', 'Emerging Market', 'Emerging Mrkts', 'International', 'Global']
        for category in exclude_categories:
            df_filtered = df_filtered[~df_filtered['lipper_class_name'].str.contains(category, case=False, na=False)]
            df_filtered = df_filtered[~df_filtered['fund_name'].str.contains(category, case=False, na=False)]

        # Keep observations for 2020 only
        df_filtered = df_filtered[df_filtered['caldt'].dt.year == 2020]

        # Drop duplicates in terms of fund identifier: crsp_portno
        df_filtered = df_filtered.drop_duplicates(subset=['crsp_portno'])
            
        # Save processed data
        # please keep this
        self.fund_summary_df = df_filtered

        df = df_filtered
        
        # return the following lists for grading
        portnos_list = df['crsp_portno'].dropna().astype(int).tolist()
        etftickers_list = df['ticker'].dropna().tolist()
        return portnos_list, etftickers_list

    def clean_shares(self, filename):
        # for this part, you work with crsp_q_mutualfunds_shares_2020.csv
        # Import shares data: CRSP monthly stock file; shrout (Number of Shares Outstanding in thousands). Query ETF tickers above, 2019m12 - 2021m1 
        # the data is also available on Canvas
        # Load dataset
        shares_df = pd.read_csv('crsp_q_mutualfunds_shares_2020.csv', parse_dates=['date'])

        # Sort by PERMNO and date to ensure the calculation is done in chronological order for each ETF
        shares_df.sort_values(by=['PERMNO', 'date'], inplace=True)

        # Calculate proportional change in shares outstanding from month to month
        shares_df['etf_shares_change'] = shares_df.groupby('PERMNO')['SHROUT'].pct_change()

        # Generate year and month variables for merging later
        shares_df['year'] = shares_df['date'].dt.year
        shares_df['month'] = shares_df['date'].dt.month

        # Save processed data
        # please keep this
        self.shares_df = shares_df

        # return the following lists for grading
        etf_shares_change_list = shares_df['etf_shares_change'].dropna().to_list()
        return etf_shares_change_list

    def impute_baskets(self, filename):
        # for this part, you work with crsp_q_mutualfunds_holdings_2020.csv, which you can download with the codes below
        '''
        df_query_holdings = "SELECT * FROM crsp.holdings WHERE (report_dt BETWEEN '2019-12-01' AND '2021-01-31')"
        merged_df = db.raw_sql(df_query_holdings)
        merged_df.to_csv(f"{data_path}/crsp_q_mutualfunds_holdings_2020.csv", index=False)
        '''
        impute_basket_df = pd.read_csv(filename)

        # handle any missing cusips: Replace missing cusips with name of security
        # Replace missing CUSIPs with the name of security
        impute_basket_df['cusip'].fillna(impute_basket_df['security_name'], inplace=True)
        
        # Sort by ETF (crsp_portno) and report date to ensure chronological processing
        impute_basket_df.sort_values(by=['crsp_portno', 'report_dt'], inplace=True)
        
        # Calculate changes in the number of shares held of a given security by an ETF across months
        impute_basket_df['shares_change'] = impute_basket_df.groupby(['crsp_portno', 'cusip'])['nbr_shares'].diff()

        # impute (intensive margin) basket: Impute intensive margin basket by taking changes in the number of shares held of a given bond (cusip) by an ETF (portno) across months
        impute_basket_df['imputed_basket_weight_level'] = impute_basket_df.groupby(['crsp_portno', 'report_dt'])['shares_change'].transform(lambda x: abs(x) / abs(x).sum())

        # Track number of bonds in portfolio and baskets
        # Calculate the number of bonds held in basket and holdings by each ETF in each month
        portfolio_num_sec = impute_basket_df.groupby(['crsp_portno', 'report_dt'])['cusip'].nunique().reset_index(name='portfolio_num_sec')
        impute_basket_df = pd.merge(impute_basket_df, portfolio_num_sec, on=['crsp_portno', 'report_dt'], how='left')
        basket_num_sec = impute_basket_df[impute_basket_df['shares_change'] != 0].groupby(['crsp_portno', 'report_dt'])['cusip'].nunique().reset_index(name='basket_num_sec')
        impute_basket_df = pd.merge(impute_basket_df, basket_num_sec, on=['crsp_portno', 'report_dt'], how='left')

        # Fill NaN values for 'imputed_basket_weight_level', 'portfolio_num_sec', and 'basket_num_sec' where no changes occurred
        impute_basket_df['imputed_basket_weight_level'].fillna(0, inplace=True)
        impute_basket_df['portfolio_num_sec'].fillna(0, inplace=True)
        impute_basket_df['basket_num_sec'].fillna(0, inplace=True)

        # Save processed data
        # please keep this
        self.impute_basket_df = impute_basket_df

        # return the following lists for grading
        portfolio_num_sec_list = impute_basket_df['portfolio_num_sec'].dropna().to_list()
        basket_num_sec_list = impute_basket_df['basket_num_sec'].dropna().to_list()
        return portfolio_num_sec_list, basket_num_sec_list
    
    
    def merge_summary_shares(self):
        fundsummary_df = self.fund_summary_df
        impute_basket_df = self.impute_basket_df
        shares_df = self.shares_df

        # Merge data from step 1 and step 3, based on 'crsp_portno'
        fundsummary_df['crsp_portno'] = fundsummary_df['crsp_portno'].astype(float)
        merged_df = pd.merge(fundsummary_df, impute_basket_df, on='crsp_portno', how='inner')

        # Before merging with shares_df, ensure there's a common 'year' and 'month' in merged_df
        merged_df['report_dt'] = pd.to_datetime(merged_df['report_dt'])
        merged_df['year'] = merged_df['report_dt'].dt.year
        merged_df['month'] = merged_df['report_dt'].dt.month

        ## Checks
        # Retrieve all column names
        columns = merged_df.columns

        # Print all column names
        print("Columns in merged_df:", list(columns))

        # Rename 'ticker_x' to 'ticker' and drop 'ticker_y'
        merged_df.rename(columns={'ticker_x': 'ticker'}, inplace=True)
        merged_df.drop(columns=['ticker_y'], inplace=True)

        shares_df = shares_df.rename(columns={'TICKER': 'ticker'})

        # Filter out rows with missing values in the join columns
        merged_df = merged_df.dropna(subset=['ticker', 'year', 'month'])
        shares_df = shares_df.dropna(subset=['ticker', 'year', 'month'])

        '''
        ## Merge to ETF Shares from step 2, based on ticker-month-year
        # Perform the merge in chunks due to huge file size
        chunk_size = 200000  # Adjust the chunk size as needed
        merged_chunks = []

        for chunk_start in range(0, shares_df.shape[0], chunk_size):
            chunk_end = min(chunk_start + chunk_size, shares_df.shape[0])
            shares_chunk = shares_df[chunk_start:chunk_end]
            merged_chunk = pd.merge(merged_df, shares_chunk,
                                    on=['ticker', 'year', 'month'], how='inner')
            merged_chunks.append(merged_chunk)

        # Concatenate the merged chunks
        merged_df = pd.concat(merged_chunks, ignore_index=True)
        '''

        merged_df = pd.merge(merged_df, shares_df, on=['ticker', 'year', 'month'], how='inner')    

        # Save processed data
        # please keep this
        self.merged_df = merged_df
        
        # return the following for grading
        return merged_df.head(5)

    def analysis(self):
        merged_df = self.merged_df
        
        # Do the following manipulations with merged_df:
            # Drop duplicates by etf-month 
            # Remove rows where number of holdings is missing or 0
            # Calculate baskets_frac
        
        # Drop duplicates by ETF-month, assuming 'ticker' and 'month' uniquely identify each ETF's monthly data
        merged_df = merged_df.drop_duplicates(subset=['ticker', 'month', 'year'])

        # Remove rows where number of holdings is missing or 0
        merged_df = merged_df[(merged_df['portfolio_num_sec'] > 0) & (merged_df['basket_num_sec'] > 0)]

        # Number of ETFs: Assuming 'ticker' uniquely identifies each ETF
        num_ETFs = merged_df['ticker'].nunique()

        # average number of bonds in portfolio 
        avg_bonds_portfolio = merged_df['portfolio_num_sec'].mean()

        # average number of bonds in basket
        avg_bonds_basket = merged_df['basket_num_sec'].mean()

        # Calculate the fraction of bonds in basket relative to portfolio for each ETF each month
        merged_df['baskets_frac'] = merged_df['basket_num_sec'] / merged_df['portfolio_num_sec']

        # average fraction of bonds in basket relative to portfolio
        avg_basket_frac = merged_df['baskets_frac'].mean()
        
        # return the following for grading
        return num_ETFs, avg_bonds_portfolio, avg_bonds_basket, avg_basket_frac
    
    def analysis_verbal(self):
        '''
        Please type your response to the question: 
        For each ETF, calculate average number of bonds in the basket and in the ETF holdings. Divide the average number of bonds in baskets by the number of bonds in the ETF holdings. What do you observe?
        '''
        reponse = "The analysis reveals that, on average, the ETF baskets contain a significant portion of the bonds held in the overall ETF portfolio. Specifically, the average fraction of bonds in the basket relative to the portfolio (avg_basket_frac) is approximately 0.5517, indicating that the baskets comprise about 55% of the total bond holdings in the ETFs. This suggests that the creation and redemption baskets play a substantial role in the monthly changes of ETF portfolio holdings. The high proportion of basket bonds relative to the total portfolio implies that the baskets are not just a small, representative sample of the ETF holdings but rather constitute a significant portion of the ETF's total bond exposure. This finding highlights the importance of understanding the composition of ETF baskets, as they can have a considerable impact on the overall portfolio structure and the liquidity of the underlying bonds."
        return reponse


In [5]:
hw = hw_ETF()

# To download the files "MonthlyReturns.csv" and "fund_summary_full.csv" from WRDS
# If you have already saved these files, you can comment out the following lines

# Homework begins here:
# Step 1: Download and clean data
# the data will be provided and data downloading will not be tested in this homework
portnos_list, etftickers_list = hw.clean_fund_summary('./crsp_q_mutualfunds_summary_2019_2023.csv')
# print(portnos_list, etftickers_list)

# Step 2: Calculate monthly changes in ETF shares
etf_shares_change = hw.clean_shares('./crsp_q_mutualfunds_shares_2020.csv')
# print(etf_shares_change)

# Step 3: Impute baskets from monthly changes in ETF holdings
portfolio_num_sec_list, basket_num_sec_list = hw.impute_baskets('./crsp_q_mutualfunds_holdings_2020.csv')
# print(portfolio_num_sec_list, basket_num_sec_list)

# Step 4: Merge variables: ETF ticker from Fund Summary; ETF Shares from Monthly Stock
df_sample = hw.merge_summary_shares()
print(df_sample)

# Step 5: Analysis
num_ETFs, avg_bonds_portfolio, avg_bonds_basket, avg_basket_frac = hw.analysis()
print(num_ETFs, avg_bonds_portfolio, avg_bonds_basket, avg_basket_frac)
response = hw.analysis_verbal()


Columns in merged_df: ['summary_period2', 'crsp_fundno', 'caldt', 'summary_period', 'nav_latest', 'nav_latest_dt', 'tna_latest', 'tna_latest_dt', 'yield', 'div_ytd', 'cap_gains_ytd', 'nav_52w_h', 'nav_52w_h_dt', 'nav_52w_l', 'nav_52w_l_dt', 'unrealized_app_dep', 'unrealized_app_dt', 'asset_dt', 'per_com', 'per_pref', 'per_conv', 'per_corp', 'per_muni', 'per_govt', 'per_oth', 'per_cash', 'per_bond', 'per_abs', 'per_mbs', 'per_eq_oth', 'per_fi_oth', 'maturity', 'maturity_dt_x', 'cusip8', 'crsp_portno', 'crsp_cl_grp', 'fund_name', 'ticker_x', 'ncusip', 'mgmt_name', 'mgmt_cd', 'mgr_name', 'mgr_dt', 'adv_name', 'open_to_inv', 'retail_fund', 'inst_fund', 'm_fund', 'index_fund_flag', 'vau_fund', 'et_flag', 'delist_cd', 'header', 'first_offer_dt', 'end_dt', 'dead_flag', 'merge_fundno', 'actual_12b1', 'max_12b1', 'mgmt_fee', 'exp_ratio', 'turn_ratio', 'fiscal_yearend', 'crsp_obj_cd', 'si_obj_cd', 'accrual_fund', 'sales_restrict', 'wbrger_obj_cd', 'policy', 'lipper_class', 'lipper_class_name', '